# Modeling with GNN-ePC-SAFT

Model combining graph neural network with ePC-SAFT


## Starting point

In [1]:
import jax.numpy as np
import jax
from jax import dlpack as jdlpack
from torch.utils import dlpack as tdlpack
from importlib import reload
import ml_pc_saft
import torch

from jax.config import config
config.update("jax_enable_x64", True)

In [20]:
reload(ml_pc_saft)

<module 'ml_pc_saft' from '/home/wildsonbbl/documents/code/ePC-SAFT/ml_pc_saft.py'>

## ML layer

In [9]:
                            #m      s       e     vola      eassoc dipm dipnum z   dielc  k    l   khb
paratorch = torch.tensor([[2.5313, 3.4608, 269.47, 0.0,     0.0,    1.04, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0],
                          [2.3827, 3.1771, 198.24, 0.032384, 2653.4, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]],
                          dtype=float, device='cuda:0', requires_grad=True)

                            #x1       x2       T        P    phase type  y
statetorch = torch.asarray([0.3607, 0.6393, 337.03, 101330.0, 1.0, 3.0, 100000.0], dtype=float, device='cuda:0')

parameters = tdlpack.to_dlpack(paratorch)
parameters = jdlpack.from_dlpack(parameters)

state = tdlpack.to_dlpack(statetorch)
state = jdlpack.from_dlpack(state)
fakebatch_para = torch.ones((64,1,1),device='cuda:0')*paratorch
fakebatch_state = torch.ones((64,1,),device='cuda:0')*statetorch

In [3]:
loss = ml_pc_saft.PCSAFTLOSS.apply

In [10]:
results = loss(fakebatch_para, fakebatch_state)
results

tensor(65.1733, device='cuda:0', dtype=torch.float64,
       grad_fn=<PCSAFTLOSSBackward>)

In [ ]:
results.backward()

In [ ]:
paratorch.grad

### PNA EPCSAFT